In [5]:
from itertools import tee

def pairwise(iterable):
    """Iterate over pairs of a sequence."""

    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

recalls = [1, 5, 4, 8, 2]
mask = [True, True, False, False, True]
for prev, curr in pairwise(zip(recalls, mask)):
    print(prev, curr)

(1, True) (5, True)
(5, True) (4, False)
(4, False) (8, False)
(8, False) (2, True)


In [10]:
def gen(n):
    i = 0
    while i < n:
        yield i
        i += 1

for i in gen(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [25]:
def masker(x, mask):
    n = 0
    while n < (len(mask) - 1):
        m = n + 1
        if mask[n] and mask[m]:
            yield x[n], x[m]
        n += 1
        
recalls = [4, 5, 6, 7, 8]
mask = [True, True, False, True, True]
inputs = list(range(1, 11))
for i in masker(recalls, mask):
    print(i)

(4, 5)
(7, 8)


In [29]:
def masker(x, mask, inputs):
    n = 0
    possible = inputs.copy()
    while n < (len(mask) - 1):
        m = n + 1
        possible.remove(x[n])
        if mask[n] and mask[m]:
            yield x[n], x[m], possible
        n += 1
        
recalls = [4, 5, 6, 7, 8, 1, 2, 9]
mask = [True, True, True, False, True, True, False, True]
inputs = list(range(1, 11))
for i in masker(recalls, mask, inputs):
    print(i)

(4, 5, [1, 2, 3, 5, 6, 7, 8, 9, 10])
(5, 6, [1, 2, 3, 6, 7, 8, 9, 10])
(8, 1, [1, 2, 3, 9, 10])


In [45]:
import numpy as np
def masker(x, from_mask, to_mask, inputs):
    n = 0
    possible = inputs.copy()
    while n < (len(x) - 1):
        m = n + 1
        if x[n] in possible:
            possible.remove(x[n])
        if from_mask[n] and to_mask[m]:
            yield x[n], x[m], possible
        n += 1
        
recalls = [4, 5, 6, 7, 8, 3, 2, 9]
#recalls = ['c', 'd', 'e', 'd', 'c', 'a', 'h', 'e']
from_mask = np.array([1, 1, 1, 0, 0, 1, 1, 0], dtype=bool)
to_mask   = np.array([1, 1, 1, 0, 0, 1, 1, 0], dtype=bool)
inputs = list(range(1, 11))
#inputs = list('abcdefgh')
for prev, curr, poss in masker(recalls, from_mask, to_mask, inputs):
    actual = curr - prev
    possible = np.subtract(poss, prev)
    print(actual, possible)

1 [-3 -2 -1  1  2  3  4  5  6]
1 [-4 -3 -2  1  2  3  4  5]
-1 [-2 -1  6  7]


In [46]:
actual = np.zeros(47)
actual[-4] += 1
actual

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [58]:
import pandas as pd
lags = np.arange(-23, 24)
actual = pd.Series(0, dtype='int', index=lags)
possible = pd.Series(0, dtype='int', index=lags)
recalls = [24, 22, 23, 16, 14, 15, 4, 5, 6, 7, 8, 3, 2, 9]
from_mask = np.array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0])
to_mask   = np.array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1])
inputs = list(range(1, 25))
for prev, curr, poss in masker(recalls, from_mask, to_mask, inputs):
    actual[curr - prev] += 1
    possible[np.subtract(poss, prev)] += 1

In [68]:
import sys
module_dir = '/Users/morton/PycharmProjects/psifr'
if module_dir not in sys.path:
    sys.path.append(module_dir)
import importlib
from psifr import fr
importlib.reload(fr)
actual = pd.Series(0, dtype='int', index=lags)
possible = pd.Series(0, dtype='int', index=lags)
masker = fr.transition_masker(recalls, inputs, from_mask, to_mask)
for prev, curr, poss in masker:
    actual[curr - prev] += 1
    possible[np.subtract(poss, prev)] += 1

In [83]:
curr = 5
valid = np.arange(10)
f = lambda x, y: x < y
valid[f(curr, valid)]

array([6, 7, 8, 9])

In [90]:
recalls = pd.Series([6, 7, 8, 9, np.nan])
recalls.astype('int')
#np.isnan(recalls)

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [86]:
importlib.reload(fr)

lags = np.arange(-23, 24)
actual = pd.Series(0, dtype='int', index=lags)
possible = pd.Series(0, dtype='int', index=lags)
recalls = [23, 21, 22, 19, 17, 18, 4, 5, 6, 7, 8, 3, 2, 9]
values = np.array([1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 2, 2, 1, 1, 1, 1, 1, 3, 3, 3])
inputs = list(range(24))
masker = fr.transition_masker(recalls, inputs, test_values=values,
                              test=lambda x, y: x == y)
for prev, curr, poss in masker:
    actual[curr - prev] += 1
    possible[np.subtract(poss, prev)] += 1

In [87]:
pd.DataFrame({'actual': actual, 'possible': possible})

,actual,possible
-23,0,0
-22,0,0
-21,0,0
-20,0,0
-19,0,1
-18,0,1
-17,0,2
-16,0,1
-15,0,1
-14,0,1


In [69]:
actual / possible

-23    0.000000
-22    0.000000
-21    0.000000
-20    0.000000
-19    0.000000
-18    0.000000
-17    0.000000
-16    0.000000
-15    0.000000
-14    0.000000
-13    0.000000
-12    0.000000
-11    0.250000
-10    0.000000
-9     0.000000
-8     0.000000
-7     0.200000
-6     0.000000
-5     0.142857
-4     0.000000
-3     0.000000
-2     0.166667
-1     0.000000
 0          NaN
 1     0.833333
 2     0.000000
 3     0.000000
 4     0.000000
 5     0.000000
 6     0.000000
 7     0.250000
 8     0.000000
 9     0.000000
 10    0.000000
 11    0.000000
 12    0.000000
 13    0.000000
 14    0.000000
 15    0.000000
 16    0.000000
 17    0.000000
 18    0.000000
 19    0.000000
 20         NaN
 21         NaN
 22         NaN
 23         NaN
dtype: float64

In [63]:
pd.DataFrame({'actual': actual, 'possible': possible}).loc[-10:10]

,actual,possible
-10,0,4
-9,0,4
-8,0,4
-7,1,5
-6,0,6
-5,1,7
-4,0,7
-3,0,7
-2,1,6
-1,0,4


In [23]:
class PairMasker:
    def __init__(self, x, mask):
        self.count = 0
        self.x = x
        self.mask = mask
    
    def __iter__(self):
        return self
    
    def 
    
    def __next__(self):
        if self.count < (len(self.x) - 1):
            n = self.count
            m = n + 1
            self.count += 1
            if self.mask[n] and self.mask[n + 1]:
                return self.x[n], self.x[n + 1]
        else:
            raise StopIteration

            
masker = PairMasker([3, 5, 7, 9, 10], [True, True, False, True, True])
myiter = iter(masker)
for x in myiter:
    print(x)

(3, 5)
None
